<a href="https://colab.research.google.com/github/matthiasdellago/nanoGPT/blob/master/exploring_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-07 10:41:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-03-07 10:41:13 (20.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
!pip install icecream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from icecream import ic

# hyperparameters
batch_size = 4#16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 2#5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 16#64
n_head = 2#4
n_layer = 2#4
dropout = 0.0
att_iterations = 2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
        self.record = False

    def set_record(self, record):
        self.record = record

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei * C**-0.5 # normalise by dimension
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        #wei = self.dropout(wei) # This might be important???!!!
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)

        # same thing with attention:
        att_out = self.attention(keys=x,queries=x,values=self.value(x))
        #check if you get the same thing. If not throw error.
        if not torch.equal(att_out, out):
          ic(att_out)
          ic(out)
          ic(torch.eq(att_out, out))
          raise Exception("Attention not implemented correctly!")

        # same thing with attention:
        # apply all the key and query matrix to one side of the scalar product
        frame = x @ self.query.weight.T @ self.key.weight
        att2_out = self.attention2(frame=frame,vec=x,values=self.value(x))
        #check if you get the same thing. If not throw error.
        if not torch.allclose(att2_out,out,atol=1e-06):
          ic(att_out)
          ic(out)
          ic(torch.eq(att_out, out))
          raise Exception("Attention not implemented correctly!")

        # make an extra forward-like function that just computes and returns the iterated attention.
        # Let's do some statistics on the attrocors in the attention layer.
        # The plan is:
        # 1. apply the attention operation to each attention vector a few times
        # 2. calculate the distances amongst all the attracted (hopefully converged vectors)
        # 3. Create a histogram of all the distances.
        # Expectation: Some distribution with a peak around (?) the average distance on
        # the unit sphere in n dimensions 
        # (https://math.stackexchange.com/questions/2366580/whats-the-average-euclidian-distance-between-two-points-on-a-unit-n-sphere) 
        # and maybe a peak at 0.
        # If there are entries close to 0 then not every attention vector gets its own attractor!

        if self.record:
          print("recording:")
          # iterate attention:
          # simple self attention with KQ but no V Matrix
          # we need to plug in the attention vectors/frame that are created by K and Q. They're not x!!
          attractors = self.attention(keys=x,queries=x,values=x)
          for _ in range(att_iterations):
            # should I plug attractors in as keys or queries?
            # does it matter?
            attractors= self.attention(keys=x,queries=x,values=x)

        return out

    def attention(self, keys, queries, values):
        assert keys.shape == queries.shape
        # assert keys.shape == values.shape
        B,T,C = keys.shape
        #ic(B,T,C)
        k = self.key(keys)   # (B,T,C)
        q = self.query(queries) # (B,T,C)
        # check that thats the same as applying the linear layer as matrices directly
        assert torch.allclose(k, keys @ self.key.weight.T)
        assert torch.allclose(q, queries @ self.query.weight.T)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) # (B, T, C) @ (B, C, T) -> (B, T, T)
        
        # express wei as q Q^T (k K^T)^T 
        #ic([i.shape for i in (queries, self.query.weight.T, self.key.weight.T, keys.T)])
        assert torch.allclose(wei, queries @ self.query.weight.T @ (keys @ self.key.weight.T).transpose(-2,-1))

        # Check if the two computations are equivalent
        assert torch.allclose(wei, queries @ self.query.weight.T @ self.key.weight @ keys.transpose(-2,-1), atol=1e-06)
        
        wei = wei * C**-0.5 # normalise by dimension
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        #wei = self.dropout(wei) # This might be important???!!!
        # perform the weighted aggregation of the values
        out = wei @ values # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

    def attention2(self, frame, vec, values):
        B,T,C = vec.shape
        
        #compute scalar product 
        wei = frame @ vec.transpose(-2,-1)
        
        wei = wei * C**-0.5 # normalise by dimension
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        # perform the weighted aggregation of the values
        out = wei @ values # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

        self.record = False

    def set_record(self, record):
        self.record = record
        for head in self.heads:
          head.set_record(record)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

        self.record = False

    def set_record(self, record):
        self.record = record
        self.sa.set_record(record)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.record = False

    def set_record(self, record):
        self.record = record
        for block in self.blocks:
          block.set_record(record)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for i in range(max_new_tokens):
            if i == max_new_tokens-1:
              self.set_record(True)
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model

m.set_record(False)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.009153 M parameters
step 0: train loss 4.3012, val loss 4.2901
step 1: train loss 4.2821, val loss 4.2749
recording:
recording:
recording:
recording:

3hphPxi ?m& &aKhYJ.X?hGHaPsd$ LI!?dr3?-svzRapP;TWZ;ddPul?FpfgZfJsreiUy?mr,ClMEa-Bra
PfPKeEWP;kOI,FTBcJrb$TvE?qKlVu&ZgTRmntLvNvmFV IkPeuFIGnsnJJv'VF?pQAkOyNDl$YSrP$-xlXCTDlBPFnz-idA3Tb.DX'FcVxVY-LrXfcnL,xYw3;kxCGRl;CKnusNrgfMOsIe3?Zalyln;&:Omyle?eEvSETt3Ik.JTvYLu'gdqok.r:ixYx
yuP?;SXC-bc,3eXdanpn-ru3iv,frbWycYcKRm.vy;cqZbZFkLSnPc,mkCufLn-cGNSn$F,OmEbfdAaYFO3:
.dOIInrIV3:rWlLoyD:n-,3r!Ir:!-roxP'rn3GDeu;
W.rtFH$UoJkoS?LXX-caVj.lE3l-F.Pi-?EfnkxVjk!kD3TBPnUVDGjoogwjL$lpyW.fk-.--VeYguTWizlL?ARwDoaWpAftTv!Y:QnC&XX3&gllN:ge ;fPXDq--JlgmQxVl,sakbtTvkyhMkq3L3sfyvfvPwhfpLEvfZ?ztkyz?va$qBFYwdpWnXFRr!i3&rYwZT,Wh;v?HEjkz!:m!p?N?p!aDhpYrVAb!Ok;PfHFA3$L,pjHD3Yfvke.ratsvjmz
FX?Tq3$UoosBc
FB
ybmyWEWJlWyXDVn3a&3hpIPWpnTplljZAgnz- HcZDyCs-k-SQ$nEaCl?iaOwyWIkK,weaYr;y,!qiFqva:eQ;:3awiFQWy-?nZqlvwzyTChwEb'BFtfFeCnZcqH;vpYir'jnGbyJfn?OgfKkePE?NS-Y$:KJPtk3nac